## Objective : We will measure performance of final model .

In [2]:
#!pip install  scikit-plot

In [1]:
#import libraries here; add more as necessary.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings("ignore")

#Models
from xgboost import XGBClassifier


from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

%load_ext autoreload
%autoreload 2

## Load Data

In [2]:
X_train = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_train_shoppersData.csv')
y_train = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_train.csv')

In [3]:
X_test = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_test_shoppersData.csv')
y_test = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_test.csv')

In [4]:
X_val = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_X_val_shoppersData.csv')
y_val = pd.read_csv(r'C:\Users\psahu\Documents\My_projects\online_shoppers\data\Transformed_y_val.csv')

In [5]:
import Functions_File as my_ff

Functions_File is successfully imported.


# Final Model

In [6]:
print(X_train.shape ,y_train.shape)

(7398, 68) (7398, 1)


In [7]:
print(X_test.shape ,y_test.shape)

(2466, 68) (2466, 1)


In [8]:
print(X_val.shape ,y_val.shape)

(2466, 68) (2466, 1)


## Before Sampling

In [9]:
y_train.value_counts()

Revenue
0          6271
1          1127
dtype: int64

In [10]:
#USing SMOTE for oversampling of Minority class
smt = SMOTE(sampling_strategy = 1)
X_train, y_train = smt.fit_sample(X_train, y_train)

## After Sampling

In [11]:
y_train.value_counts()

Revenue
1          6271
0          6271
dtype: int64

## Model Stacking

In [158]:
from sklearn.ensemble import StackingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from lightgbm import LGBMClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

In [159]:
#Here we have tried different stacking combination.
#Combination of GBM + MLP + XGB is giving us best results.

In [198]:
#Stacking

level0 = list()
#level0.append(('rfc', RandomForestClassifier()))
#level0.append(('et', ExtraTreesClassifier()))
level0.append(('gbm', GradientBoostingClassifier()))
level0.append(('NN', MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=50)))
 

# define meta learner model
level1 = XGBClassifier(colsample_bytree=0.7, gamma=0.0,learning_rate=0.05, max_depth=4,min_child_weight=3, random_state=50)

# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

#model = level1

In [199]:
model.fit(X_train,y_train)

StackingClassifier(cv=5,
                   estimators=[('gbm', GradientBoostingClassifier()),
                               ('NN',
                                MLPClassifier(alpha=1e-05,
                                              hidden_layer_sizes=(5, 2),
                                              random_state=50,
                                              solver='lbfgs'))],
                   final_estimator=XGBClassifier(colsample_bytree=0.7,
                                                 gamma=0.0, learning_rate=0.05,
                                                 max_depth=4,
                                                 min_child_weight=3,
                                                 random_state=50))

## Prediction on test set.  

In [204]:
y_pred_test = model.predict(X_test)
print("Accuracy:" , accuracy_score(y_test,y_pred_test)*100)

Accuracy: 88.3617193836172


In [205]:
confusion_matrix(y_test, y_pred_test)

array([[1875,  209],
       [  78,  304]], dtype=int64)

In [206]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred_test,  average='binary')

0.7958115183246073

In [207]:
#Compute precision, recall, F-measure and support

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93      2084
           1       0.59      0.80      0.68       382

    accuracy                           0.88      2466
   macro avg       0.78      0.85      0.80      2466
weighted avg       0.90      0.88      0.89      2466



##  Prediction on validation set 

In [208]:
y_pred_val = model.predict(X_val)
print("Accuracy:" , accuracy_score(y_val,y_pred_val)*100)

Accuracy: 87.91565287915653


In [209]:
recall_score_val=recall_score(y_val,y_pred_val,  average='binary')
recall_score_val

0.7994987468671679

In [210]:
#
X_val_probs = model.predict_proba(X_val)[:,1]
#%% Confusion metrics - validation set
confusion_matrix(y_val, y_pred_val)

array([[1849,  218],
       [  80,  319]], dtype=int64)

In [211]:

#%% Compute precision, recall, F-measure and support

from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       0.96      0.89      0.93      2067
           1       0.59      0.80      0.68       399

    accuracy                           0.88      2466
   macro avg       0.78      0.85      0.80      2466
weighted avg       0.90      0.88      0.89      2466

